# Weight perturbation implementation in python 

We will be using the iris dataset for this! Let us look at how the dataset looks like...before that lets import the libraries required.

In [1]:
import pandas as pd
import numpy as np
import random
from typing import List, Callable
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

## Reading the dataset

In [ ]:
df = pd.read_csv("C:\Users\Rohit\Desktop\Perturbation Techniques in CNNs\Perturbation-techniques-in-CNNs\data\Iris.csv")